# Modified Ochiai 4.1
Fake failing tests set: Tests that cover more lines of each ST method. This process is done for each stack trace entry in the top 5, once at a time. Having the Ochiai scores for each of them, a final ranking is calculated via an average of the scores, same weigths for all of them

In [4]:
import os
from my_secrets import base_path


paths_dict=  {
    "ochiai_scores_4.0_files": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiScores", "modifiedOchiai4.0"),
    "output_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiScores", "modifiedOchiai4.1"),
    "tests_analysis_results":  os.path.join(base_path,"DeepDiveBugReportsWithLogs",  "data", "rq1_results.json"),
}

## Getting the final Ochiai scores file with the weighted average

In [6]:
%run ../utils.py

weights_list = [1, 1, 1, 1, 1]
tests_analysis_results  = json_file_to_dict(paths_dict["tests_analysis_results"])
bugs = get_list_of_bugs_with_coverage(tests_analysis_results)

for bug in bugs:
    project, bug_id = bug.split("_")
    ochiai_scores_folder = os.path.join(paths_dict["ochiai_scores_4.0_files"], project, bug_id)
    entries_file_scores = get_json_files(ochiai_scores_folder)
    ochiai_scores_data = {}
    sum_weights = 0
    for entry_number, entry_file in enumerate(entries_file_scores):
        entry_data = json_file_to_dict(entry_file)
        weight = weights_list[entry_number]
        sum_weights += weight
        for key in entry_data.keys():
            if key not in ochiai_scores_data.keys():
                ochiai_scores_data[key] = weight * entry_data[key]
            else:
                ochiai_scores_data[key] += weight * entry_data[key]
    for key in ochiai_scores_data.keys():
        ochiai_scores_data[key] = ochiai_scores_data[key]/sum_weights

    dict_to_json_file(os.path.join(paths_dict["output_path"], project, bug_id + ".json"), ochiai_scores_data)

print("Execution completed")

Execution completed
